<a href="https://colab.research.google.com/github/kavyakapoor200/Reddit-Bot/blob/main/reddit_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Reddit Posting Bot
==================

This script uses Reddit's API (via PRAW) and Perplexity's LLM API to generate and post content to various subreddits automatically.
It includes logic for active hours, cooldown tracking using a persistent JSON file, flair assignment, and generation of fallback posts.

"""

# === IMPORTS ===
import time  # To control the posting interval and cooldown wait
from datetime import datetime, timedelta  # For time-based logic
import pytz  # To handle Indian Standard Time (IST)
import os  # For environment-based configuration if needed
import praw  # Python Reddit API Wrapper
import requests  # For making API calls to Perplexity
import json  # For reading and writing cooldown data to file

# === PERPLEXITY SETUP ===
PPLX_API_KEY = "pplx-xxxxxxx"  # Your Perplexity API key
PPLX_URL = "https:"  # Perplexity chat endpoint
HEADERS = {
    "Authorization": f"Bearer {PPLX_API_KEY}",
    "Content-Type": "application/json"
}  # Headers for Perplexity API call

# === REDDIT SETUP ===
reddit = praw.Reddit(
    client_id="Z3mvLUh2xxxxx",
    client_secret="YppwtxiLypT--xxxxxxx",
    username="No-Mulbexxxxx",
    password="Kavyxxxxxx",
    user_agent="script:BotPoxxxxxxxx"
)
print("Logged in as:", reddit.user.me())

# === SUBREDDIT CONFIGURATION ===
subreddits = {
    "r/algotrading": {"instruction": "Focus on practical bot-building advice and community sharing.", "active_hours": (9, 15)},
    "r/quant": {"instruction": "Relate to mathematical modeling or statistical strategy design.", "active_hours": (19, 2)},
    "r/QuantFinance": {"instruction": "Mention how bots integrate with quant models or financial instruments.", "active_hours": (19, 2)},
    "r/HighFrequencyTrading": {"instruction": "Mention low-latency execution, high-speed APIs, or optimization tricks.", "active_hours": (20, 3)},
    "r/Python": {"instruction": "Emphasize Python code structure, APIs, and script examples.", "active_hours": (10, 18)},
    "r/learnpython": {"instruction": "Keep tone beginner-friendly, explain how bots work step-by-step.", "active_hours": (11, 17)},
    "r/Backtesting": {"instruction": "Emphasize historical data testing, risk metrics, and validation.", "active_hours": (9, 12)},
    "r/AlgoTradingIndia": {"instruction": "Include Indian context — Kite Connect, Zerodha, NSE/BSE specifics.", "active_hours": (9, 15)},
    "r/TradeBots": {"instruction": "Discuss automation logic, scheduling, error handling, and deployment.", "active_hours": (15, 20)},
    "r/NSEbets": {"instruction": "Keep it casual; include Indian retail trading context and shareable tips.", "active_hours": (10, 16)}
}

# === COOLDOWN TRACKING FILE ===
LAST_POSTED_FILE = "last_posted.json"  # Stores the last post time for each subreddit
try:
    with open(LAST_POSTED_FILE, "r") as f:
        last_posted = {sr: datetime.fromisoformat(t) for sr, t in json.load(f).items()}
except FileNotFoundError:
    last_posted = {}

# === UTILITY FUNCTIONS ===
def get_current_ist_time():
    """Returns the current time in IST."""
    return datetime.now(pytz.timezone('Asia/Kolkata'))

def is_within_active_hours(start, end, hour):
    """Checks if the current hour is within the active posting hours."""
    return start <= hour < end if start <= end else (hour >= start or hour < end)

def can_post(subreddit):
    """Determines whether the bot can post in the subreddit based on time and cooldown."""
    now = get_current_ist_time()
    hour = now.hour
    start, end = subreddits.get(subreddit, {}).get("active_hours", (0, 24))
    last_time = last_posted.get(subreddit)
    return is_within_active_hours(start, end, hour) and (last_time is None or (now - last_time) >= timedelta(hours=1))

def update_last_posted(subreddit):
    """Updates the last posted time for a subreddit and saves it to JSON."""
    last_posted[subreddit] = get_current_ist_time()
    with open(LAST_POSTED_FILE, "w") as f:
        json.dump({sr: t.isoformat() for sr, t in last_posted.items()}, f)

def clean_output(text):
    """Cleans Perplexity output by removing references like [1], [2], etc."""
    import re
    text = re.sub(r'\[\d+\]', '', text)
    return "\n".join([line.strip() for line in text.splitlines() if line.strip()])

def check_removal_status(submission):
    """Waits and checks if the post was removed by Reddit or a moderator."""
    print(" Waiting 20s for Reddit filter check...")
    time.sleep(20)
    try:
        if submission.removed_by_category: return f"Removed by Reddit: {submission.removed_by_category}"
        if submission.mod_reason_by: return "Removed manually by mod"
        return "Post is live"
    except Exception as e:
        return f"Error checking: {str(e)}"

# === PERPLEXITY LLM WRAPPER ===
def call_perplexity(prompt):
    """Sends a prompt to the Perplexity API and returns the cleaned response."""
    payload = {
        "model": "sonar-pro",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7,
        "max_tokens": 800
    }
    try:
        resp = requests.post(PPLX_URL, headers=HEADERS, json=payload)
        return clean_output(resp.json()['choices'][0]['message']['content'])
    except Exception as e:
        return f"Error: {e}"

# === STRATEGY-BASED POST GENERATION ===
def generate_from_strategy(strategy):
    """Generates subreddit, title, and post content for a given strategy using Perplexity."""
    subreddit_prompt = f"""Suggest the most appropriate Indian trading subreddit for this strategy:
{strategy}
Respond only with subreddit name like r/AlgoTradingIndia or r/QuantFinance."""
    subreddit = call_perplexity(subreddit_prompt).split()[0]

    title_prompt = f"""Write a professional, short Reddit title (max 80 characters) for this strategy:
{strategy}"""
    title = call_perplexity(title_prompt).splitlines()[0]

    post_prompt = f"""You're an Indian retail trader actively experimenting with algorithmic strategies.
Write a short (~50 words) Reddit post about the following strategy:
{strategy}
Make it sound real and based on your experience — briefly mention your results or what you observed. Keep the tone friendly, professional, and collaborative. Don’t include any links, emojis, or headings.
End with a community-friendly line like:
“Has anyone else explored this kind of setup?” or “Happy to hear your thoughts or improvements.”"""

    post = call_perplexity(post_prompt)

    return subreddit, title, post

# === FALLBACK POST GENERATION ===
def generate_fallback_post(subreddit, instruction):
    """Generates a post for fallback when user doesn’t input a strategy."""
    topic_prompt = f"""Generate a relevant Reddit topic idea (not title) for the subreddit {subreddit}.
Instruction: {instruction}. Make sure it's related to algo trading or automation."""
    topic = call_perplexity(topic_prompt)

    title_prompt = f"""Write a concise, professional Reddit post title (max 80 chars) for the topic:
{topic}"""
    title = call_perplexity(title_prompt).splitlines()[0]

    post_prompt = f"""You're an experienced user posting in {subreddit}.
Write a short (around ~50 words) Reddit post on the topic:
{topic}
Make it sound helpful, collaborative, and professional. Mention what you've explored or observed related to the topic. Avoid links, emojis, or hashtags. Use one natural paragraph.
Close with a soft engagement line like:
“Would love to hear how others are approaching this,” or “Open to ideas if anyone’s done something similar.”"""

    post = call_perplexity(post_prompt)

    return title, post

# === REDDIT POST FUNCTION ===
def post_to_subreddit(subreddit, title, post_body, force_post=False):
    """Posts the given title and body to the specified subreddit. Adds flair if needed."""
    if not force_post and not can_post(subreddit):
        print(f"⏳ Cooldown or inactive hours for {subreddit}")
        return
    try:
        sr = reddit.subreddit(subreddit.replace("r/", ""))
        link_flairs = list(sr.flair.link_templates)
        flair_id = None
        if link_flairs:
            flair_id = link_flairs[0]["id"]
        submission = sr.submit(title=title, selftext=post_body, flair_id=flair_id)
        update_last_posted(subreddit)
        print(f"\n✅ Posted to {subreddit}")
        print(f"📌 Title: {title}")
        print(f"📝 Body:\n{post_body}")
        print(check_removal_status(submission))
    except Exception as e:
        print(f"❌ Posting error: {e}")


# === MAIN BOT LOOP ===
if __name__ == "__main__":
    """Main loop to take user input or automatically generate fallback posts."""
    print("Bot started... Press Ctrl+C to stop.\n")

    while True:
        user_input = input("Do you have a strategy to post? (yes/no): ").strip().lower()

        if user_input == "yes":
            strategy = input("Enter your trading strategy: ").strip()
            if strategy:
                try:
                    subreddit, title, post = generate_from_strategy(strategy)
                    post_to_subreddit(subreddit, title, post, force_post=True)  # ⬅️ FORCE POST FOR STRATEGIES
                except Exception as e:
                    import traceback
                    traceback.print_exc()
                    print(f"Error posting strategy: {e}")
            else:
                print("No input provided.")
        else:
            for sr, config in subreddits.items():
                try:
                    if can_post(sr):
                        title, post = generate_fallback_post(sr, config["instruction"])
                        post_to_subreddit(sr, title, post)  # ⬅️ fallback respects cooldown
                        time.sleep(10)
                    else:
                        print(f"⏳ Skipping {sr} (not in active hours or cooldown)")
                except Exception as e:
                    import traceback
                    traceback.print_exc()
                    print(f"Error posting to {sr}: {e}")

        print("\n🕒 Sleeping for 15 minutes before next check...\n")
        time.sleep(900)



It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Logged in as: No-Mulberry9473
Bot started... Press Ctrl+C to stop.

Do you have a strategy to post? (yes/no): yes
Enter your trading strategy: Breakout trading on BankNifty using 20EMA + CPR + Volume Surge on 15-min chart


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




✅ Posted to r/IndianStreetBets
📌 Title: BankNifty Breakout Strategy: 20EMA, CPR & Volume Surge on 15m Chart
📝 Body:
I’ve been testing a breakout strategy on BankNifty using the 20EMA, CPR, and volume surge on the 15-min chart. Entries on strong volume breakouts above CPR with EMA confirmation have shown good momentum, though fakeouts still happen, especially in choppy sessions. Proper stop-loss and time filters help. Happy to hear your thoughts or improvements.
 Waiting 20s for Reddit filter check...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Removed by Reddit: reddit

🕒 Sleeping for 15 minutes before next check...



KeyboardInterrupt: 

In [ ]:
!pip install praw google-generativeai pytz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.6 MB/s eta 0:00:00


## start



In [ ]:
"""
Reddit Subreddit Fallback Auto-Poster
=====================================

This script auto-generates and posts one Reddit post every 30 minutes
to a list of preconfigured subreddits. It uses the Perplexity LLM
API to create title and content based on subreddit-specific prompts.

Cooldowns and active hours are ignored for this version.
"""

# === IMPORTS ===
import time
import praw
import requests
import json
import pytz
from datetime import datetime

# === PERPLEXITY SETUP ===
PPLX_API_KEY = "pplx-"
PPLX_URL = "https:"
HEADERS = {
    "Authorization": f"Bearer {PPLX_API_KEY}",
    "Content-Type": "application/json"
}

# === REDDIT SETUP ===
reddit = praw.Reddit(
    client_id="Z3mv",
    client_secret="YppwtxiLy",
    username="No-Mu3",
    password="Kavy",
    user_agent="script:BotPo"
)
print(" Logged in as:", reddit.user.me())

# === SUBREDDIT CONFIGURATION ===
subreddits = {
    "r/algotrading": {"instruction": "Focus on practical bot-building advice and community sharing."},
    "r/quant": {"instruction": "Relate to mathematical modeling or statistical strategy design."},
    "r/QuantFinance": {"instruction": "Mention how bots integrate with quant models or financial instruments."},
    "r/HighFrequencyTrading": {"instruction": "Mention low-latency execution, high-speed APIs, or optimization tricks."},
    "r/Python": {"instruction": "Emphasize Python code structure, APIs, and script examples."},
    "r/learnpython": {"instruction": "Keep tone beginner-friendly, explain how bots work step-by-step."},
    "r/Backtesting": {"instruction": "Emphasize historical data testing, risk metrics, and validation."},
    "r/AlgoTradingIndia": {"instruction": "Include Indian context — Kite Connect, Zerodha, NSE/BSE specifics."},
    "r/TradeBots": {"instruction": "Discuss automation logic, scheduling, error handling, and deployment."},
    "r/NSEbets": {"instruction": "Keep it casual; include Indian retail trading context and shareable tips."}
}


def get_current_ist_time():
    """Returns the current IST time as datetime object."""
    return datetime.now(pytz.timezone("Asia/Kolkata"))


def clean_output(text):
    """Cleans output text by removing citations like [1], [2], etc."""
    import re
    text = re.sub(r'\[\d+\]', '', text)
    return "\n".join([line.strip() for line in text.splitlines() if line.strip()])


def call_perplexity(prompt):
    """Sends a prompt to Perplexity API and returns the cleaned result."""
    payload = {
        "model": "sonar-pro",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7,
        "max_tokens": 800
    }
    try:
        resp = requests.post(PPLX_URL, headers=HEADERS, json=payload)
        return clean_output(resp.json()['choices'][0]['message']['content'])
    except Exception as e:
        return f"Error: {e}"


def generate_fallback_post(subreddit, instruction):
    """
    Generates a fallback post (title + body) for a given subreddit
    based on its instruction using Perplexity.
    """
    topic_prompt = f"""Generate a relevant Reddit topic idea (not title) for the subreddit {subreddit}.
Instruction: {instruction}. Make sure it's related to algo trading or automation."""
    topic = call_perplexity(topic_prompt)

    title_prompt = f"""Write a concise, professional Reddit post title (max 80 chars) for the topic:
{topic}"""
    title = call_perplexity(title_prompt).splitlines()[0]

    post_prompt = f"""You're an experienced user posting in {subreddit}.
Write a short (around ~50 words) Reddit post on the topic:
{topic}
Make it sound helpful, collaborative, and professional. Mention what you've explored or observed related to the topic. Avoid links, emojis, or hashtags. Use one natural paragraph.
Close with a soft engagement line like:
“Would love to hear how others are approaching this,” or “Open to ideas if anyone’s done something similar.”"""
    post = call_perplexity(post_prompt)

    return title, post


def post_to_subreddit(subreddit, title, post_body):
    """
    Posts to the given subreddit with optional flair (first flair by default).
    Prints confirmation or any error if posting fails.
    """
    try:
        sr = reddit.subreddit(subreddit.replace("r/", ""))
        flair_id = None
        link_flairs = list(sr.flair.link_templates)
        if link_flairs:
            flair_id = link_flairs[0]["id"]
        submission = sr.submit(title=title, selftext=post_body, flair_id=flair_id)

        print(f"\n Posted to {subreddit}")
        print(f" Title: {title}")
        print(f" Post:\n{post_body}")
    except Exception as e:
        print(f" Posting error: {e}")


# === MAIN LOOP ===
if __name__ == "__main__":
    print("\n Reddit Fallback Bot started...\n")

    sub_list = list(subreddits.items())
    index = 0

    while True:
        sr, config = sub_list[index % len(sub_list)]
        index += 1

        print(f"\n {get_current_ist_time().strftime('%Y-%m-%d %H:%M:%S')} — Generating for {sr}...")

        try:
            title, post = generate_fallback_post(sr, config["instruction"])
            post_to_subreddit(sr, title, post)
        except Exception as e:
            import traceback
            traceback.print_exc()
            print(f" Error during post generation or submission: {e}")

        print("\n Sleeping for 30 minutes...\n")
        time.sleep(1800)


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



 Logged in as: No-Mulberry9473

 Reddit Fallback Bot started...


 2025-07-17 00:47:19 — Generating for r/algotrading...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




 Posted to r/algotrading
 Title: Share Your Step-by-Step Bot-Building Process: APIs, Code, Pitfalls & Live Trading Tips
 Post:
Looking for a candid discussion on what actually works (and what doesn’t) when integrating live market data APIs into custom trading bots—especially using Python/Pandas. I’ve found that while setting up a robust environment and choosing a broker with reliable API access is crucial, the real challenges often come during live execution: managing order flow, handling exceptions, and programmatically enforcing risk controls like stop-loss and portfolio safety. I’d appreciate seeing community members share their step-by-step workflows, code snippets, and lessons learned—especially around debugging, backtesting-to-live transitions, and ensuring system reliability under real market conditions. Would love to hear how others are approaching this.

 Sleeping for 30 minutes...



KeyboardInterrupt: 

In [ ]:
"""
Reddit Strategy-Based Auto-Poster
=================================

This script uses the Perplexity LLM to generate and post one Reddit post
every 30 minutes based on a predefined list of trading strategies.
The model predicts the appropriate subreddit, generates a title,
and writes a short post.

"""

# === IMPORTS ===
import time
import praw
import requests
import pytz
from datetime import datetime
import re

# === PERPLEXITY SETUP ===
PPLX_API_KEY = "pplx-"
PPLX_URL = "https:"
HEADERS = {
    "Authorization": f"Bearer {PPLX_API_KEY}",
    "Content-Type": "application/json"
}

# === REDDIT SETUP ===
reddit = praw.Reddit(
    client_id="Z3mvLU",
    client_secret="Yppwtxi",
    username="No-Mul",
    password="Kav",
    user_agent="script:BotPo"
)
print(" Logged in as:", reddit.user.me())


def get_current_ist_time():
    """Returns the current IST time as datetime object."""
    return datetime.now(pytz.timezone("Asia/Kolkata"))


def clean_output(text):
    """Cleans output text by removing citations like [1], [2], etc."""
    text = re.sub(r'\[\d+\]', '', text)
    return "\n".join([line.strip() for line in text.splitlines() if line.strip()])


def call_perplexity(prompt):
    """Sends a prompt to Perplexity API and returns the cleaned result."""
    payload = {
        "model": "sonar-pro",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7,
        "max_tokens": 800
    }
    try:
        resp = requests.post(PPLX_URL, headers=HEADERS, json=payload)
        return clean_output(resp.json()['choices'][0]['message']['content'])
    except Exception as e:
        return f"Error: {e}"


def generate_from_strategy(strategy):
    """
    Given a trading strategy (string), predicts subreddit, generates title and body.
    """
    subreddit_prompt = f"""Suggest the most appropriate Indian trading subreddit for this strategy:
{strategy}
Respond only with subreddit name like r/AlgoTradingIndia or r/QuantFinance."""
    subreddit = call_perplexity(subreddit_prompt).split()[0]

    title_prompt = f"""Write a professional, short Reddit title (max 80 characters) for this strategy:
{strategy}"""
    title = call_perplexity(title_prompt).splitlines()[0]

    post_prompt = f"""You're an Indian retail trader actively experimenting with algorithmic strategies.
Write a short (~50 words) Reddit post about the following strategy:
{strategy}
Make it sound real and based on your experience — briefly mention your results or what you observed. Keep the tone friendly, professional, and collaborative. Don’t include any links, emojis, or headings.
End with a community-friendly line like:
“Has anyone else explored this kind of setup?” or “Happy to hear your thoughts or improvements.”"""
    post = call_perplexity(post_prompt)

    return subreddit, title, post


def post_to_subreddit(subreddit, title, post_body):
    """
    Posts to the given subreddit with flair (if available).
    Prints confirmation or error.
    """
    try:
        sr = reddit.subreddit(subreddit.replace("r/", ""))
        flair_id = None
        link_flairs = list(sr.flair.link_templates)
        if link_flairs:
            flair_id = link_flairs[0]["id"]
        submission = sr.submit(title=title, selftext=post_body, flair_id=flair_id)

        print(f"\n Posted to {subreddit}")
        print(f" Title: {title}")
        print(f" Post:\n{post_body}")
    except Exception as e:
        print(f" Posting error: {e}")


# === STRATEGY LIST ===
strategies = [
    "Options writing based on high IV percentile and neutral delta exposure using weekly expiry Nifty options",
        "Momentum scalping using Heikin-Ashi candles and MACD crossover on BankNifty 5-minute chart"
]


# === MAIN LOOP ===
if __name__ == "__main__":
    print("\n Reddit Strategy Poster started...\n")

    index = 0
    while True:
        strategy = strategies[index % len(strategies)]
        index += 1

        print(f"\n {get_current_ist_time().strftime('%Y-%m-%d %H:%M:%S')} — Posting for strategy:")
        print(f" {strategy}")

        try:
            subreddit, title, post = generate_from_strategy(strategy)
            post_to_subreddit(subreddit, title, post)
        except Exception as e:
            import traceback
            traceback.print_exc()
            print(f" Error during post generation or submission: {e}")

        print("\n Sleeping for 30 minutes...\n")
        time.sleep(1800)


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



 Logged in as: No-Mulberry9473

 Reddit Strategy Poster started...


 2025-07-17 01:09:42 — Posting for strategy:
 Options writing based on high IV percentile and neutral delta exposure using weekly expiry Nifty options


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




 Posted to r/IndianStreetBets
 Title: High IV Percentile, Neutral Delta: Weekly Nifty Options Writing Strategy
 Post:
I’ve been testing a strategy of **writing weekly Nifty options** when the **IV percentile is high** and keeping my **delta exposure neutral**. The elevated premiums have boosted my returns, but I did notice some wild swings when volatility suddenly dropped or spiked. Has anyone else explored this kind of setup? Happy to hear your thoughts or improvements.

 Sleeping for 30 minutes...



KeyboardInterrupt: 

In [ ]:
import praw

# === Reddit API Credentials ===
reddit = praw.Reddit(
    client_id="Z3mvLUh",
    client_secret="Yppw",
    username="No-Mu",
    password="Kavy",
    user_agent="script:ImageP"
)

print(" Logged in as:", reddit.user.me())

# === Your Inputs ===
subreddit_name = "r/NSEbets"  # ← Change if needed
image_path = "/content/algotrading.jpg"  # ← Replace this with your image path
title = "MACD crossover snapshot from BankNifty 5-min"  # ← Your image title

# === Post Image ===
try:
    subreddit = reddit.subreddit(subreddit_name)
    submission = subreddit.submit_image(title=title, image_path=image_path)
    print(" Posted successfully!")
    print(" URL:", submission.url)
except Exception as e:
    print(" Error while posting:", e)


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



 Logged in as: No-Mulberry9473


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



 Posted successfully!
 URL: https://i.redd.it/ysf397qkladf1.jpeg
